In [1]:
import pandas as pd
import numpy as np
import cv2, os
import torch
import torch.nn as nn
import seaborn as sns
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as dsets
from sklearn.metrics import classification_report, confusion_matrix, precision_score,recall_score,f1_score, roc_curve, roc_auc_score, matthews_corrcoef
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
from torch.utils.data import Dataset, DataLoader
import torch_directml
import shutil
from scores import score_in_details
torch.manual_seed(240)

In [2]:
dml = torch_directml.device()

In [3]:
transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize(size = (224, 224)),
        transforms.ToTensor(),
        ])

In [4]:
class MyDataset(Dataset):
    def __init__(self, ds, transform):
        self.ds = ds
        self.transform = transform
                
    def __len__(self):
        return len(self.ds)

    def __getitem__(self, idx):            
        image_path = self.ds[idx][0]
        label = self.ds[idx][1]
        img = cv2.imread(image_path)
        img = self.transform(img)
            
        return {"image": img,
                "labels": label,
                "path": image_path}

In [5]:
dataset_path = "../splitted"
test_dataset = dsets.ImageFolder(root=f'{dataset_path}/test/')
test_ds = MyDataset(ds=test_dataset.imgs, transform=transform)

In [6]:
test_loader = DataLoader(test_ds, batch_size=20, drop_last=True, shuffle=False)

In [7]:
def create_model():
    model = torchvision.models.densenet121(weights="IMAGENET1K_V1")
    
    model.classifier = nn.Linear(1024, 2)
    return model.to(dml)

base_model = create_model()

In [8]:
model_path = "../trained-models/densenet-121/densenet121.pt"
base_model.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')), strict=False)

<All keys matched successfully>

In [12]:
def get_predictions():
    softmax = nn.Softmax(dim=1)
    base_model.eval()
    predictions = []
    real_values = []
    pred_probas = []
    misclassified = []

    with torch.no_grad():
        for batch in tqdm(test_loader):
            inputs = batch['image'].to(dml)
            labels = batch['labels']
            path = batch["path"]
            
            outputs = softmax(base_model(inputs))
            
            probas, preds = torch.max(outputs, dim = 1)
                
            preds = preds.cpu().numpy()
            labels = labels.cpu().numpy()
            probas = probas.cpu().numpy()

            predictions.extend(preds)
            real_values.extend(labels)
            pred_probas.extend(probas)
            
            for ind, (actual, pred) in enumerate(zip(labels, preds)):
                if actual != pred:
                    misclassified.append((path[ind], actual, pred))
            
    return predictions, real_values, pred_probas, misclassified

In [13]:
pred, real, probas, misclassified = get_predictions()
print(classification_report(real, pred, digits=6))

  0%|          | 0/943 [00:00<?, ?it/s]

              precision    recall  f1-score   support

           0   0.964155  0.961413  0.962782      4924
           1   0.986380  0.987371  0.986875     13936

    accuracy                       0.980594     18860
   macro avg   0.975267  0.974392  0.974829     18860
weighted avg   0.980577  0.980594  0.980585     18860



In [14]:
score_in_details(pred=pred, real=real, probas=probas)

Accuracy: 98.0593849416755%
Precision: 0.9805773514321154
Recall: 0.9805938494167551
F1-score: 0.9805849071764835
MCC-score: 0.9496591154919901
ROC AUC score: 0.7562224078087971




In [12]:
dest = "../images/misclassified-new/densenet-121"
for path, actual, pred in tqdm(misclassified):
    img_name = path.split("\\")[-1].split(".")[0]
    ext = path.split(".")[-1]
    shutil.copy(path, os.path.join(dest, f"{img_name}_{actual.item()}_{pred.item()}.{ext}"))

  0%|          | 0/352 [00:00<?, ?it/s]